## Project 3: Cross-Validating Classification Models ## 

I will be running cross-validation on four classification models under two scenearios for this project. 

The four classifiers are Logistic Regression, Linear Discriminant Analysis, Quadratic Discriminant Analysis, and K-Nearest Neighbor. 

K-fold cross validation is used, with 5 folds selected as the k-value. 

Each model will be run using the full dataset for training and testing (100% data scenario) as well as with the dataset split into 50% for training and 50% for testing (50% data scenario). 5-fold cross validation will be run for each model and scenario.

In [2]:
#begin by importing all needed modules and functions

import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf


from sklearn import neighbors
from sklearn.model_selection import KFold 
from sklearn.model_selection import cross_val_score
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis 
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report, precision_score, accuracy_score
from sklearn.utils import shuffle

from time import time
import timeit  #imports timeit module

%matplotlib inline

## Part 1: Logistic Regression

In [3]:
#Read in dataset, check for empty rows and drop them

df = pd.read_csv('SRER_final_bi_v1.csv') 
df.info()
df.isnull().sum()
df.dropna(axis=0,inplace=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4339 entries, 0 to 4338
Data columns (total 41 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   OID_          4339 non-null   int64  
 1   Id            4339 non-null   int64  
 2   gridcode      4339 non-null   int64  
 3   Shape_Length  4339 non-null   float64
 4   Shape_Area    4339 non-null   float64
 5   CH_min        4339 non-null   float64
 6   CH_max        4339 non-null   float64
 7   CH_mean       4339 non-null   float64
 8   CH_range      4339 non-null   float64
 9   CH_med        4339 non-null   float64
 10  CC_min        4339 non-null   float64
 11  CC_max        4339 non-null   float64
 12  CC_mean       4339 non-null   float64
 13  CC_range      4339 non-null   float64
 14  CC_med        4339 non-null   float64
 15  CD_min        4339 non-null   float64
 16  CD_max        4339 non-null   float64
 17  CD_mean       4339 non-null   float64
 18  CD_range      4339 non-null 

In [4]:
#check for null values
df.isnull().sum()

OID_            0
Id              0
gridcode        0
Shape_Length    0
Shape_Area      0
CH_min          0
CH_max          0
CH_mean         0
CH_range        0
CH_med          0
CC_min          0
CC_max          0
CC_mean         0
CC_range        0
CC_med          0
CD_min          0
CD_max          0
CD_mean         0
CD_range        0
CD_med          0
ARVI_min        0
ARVI_max        0
ARVI_mean       0
ARVI_range      0
ARVI_med        0
EVI_min         0
EVI_max         0
EVI_mean        0
EVI_range       0
EVI_med         0
NDVI_min        0
NDVI_max        0
NDVI_mean       0
NDVI_range      0
NDVI_med        0
SAVI_min        0
SAVI_max        0
SAVI_mean       0
SAVI_range      0
SAVI_med        0
Veg_Class       0
dtype: int64

In [4]:
#preview dataset and determine shape
print(df.head())
print(df.shape) 

   OID_  Id  gridcode  Shape_Length  Shape_Area  CH_min  CH_max   CH_mean  \
0     1   2         2          41.6       17.41    0.00    0.93  0.190714   
1     2   3         3          31.4        5.05    0.00    0.21  0.045000   
2     3   5         5          33.4        4.84    0.00    0.02  0.010000   
3     4   6         6          32.2       14.20    0.00    0.02  0.011667   
4     5   7         7          28.8       10.73    0.01    0.02  0.013750   

   CH_range  CH_med  ...  NDVI_max  NDVI_mean  NDVI_range  NDVI_med  SAVI_min  \
0      0.93   0.015  ...  0.665698   0.476709    0.382507  0.507514  0.187793   
1      0.21   0.015  ...  0.526375   0.406745    0.306644  0.433791  0.176977   
2      0.02   0.010  ...  0.208300   0.181575    0.076994  0.205117  0.109678   
3      0.02   0.010  ...  0.341480   0.207390    0.234715  0.197327  0.092876   
4      0.01   0.010  ...  0.457103   0.247391    0.334749  0.228112  0.094757   

   SAVI_max  SAVI_mean  SAVI_range  SAVI_med  Veg_

In [5]:
#separate categorical response from the rest of the data
#warnings were encountered when including CH_mean and CH_med variables for log. reg., so they were excluded from the model
formula = 'Veg_class ~ CH_max+CH_range+CC_min+CC_max+CC_mean+CC_range+CC_med+CD_min+CD_max+CD_mean+CD_range+CD_med+ARVI_min+ARVI_max+ARVI_mean+ARVI_range+ARVI_med+EVI_min+EVI_max+EVI_mean+EVI_range+EVI_med+NDVI_min+NDVI_max+NDVI_mean+NDVI_range+NDVI_med+SAVI_min+SAVI_max+SAVI_mean+SAVI_range+SAVI_med'
formula2 = 'Veg_class ~ CC_max+CC_mean+CC_range+CC_med+CD_min+CD_max+CD_mean+CD_range+CD_med+ARVI_min+ARVI_max+ARVI_mean+ARVI_range+ARVI_med+EVI_min+EVI_max+EVI_mean+EVI_range+EVI_med+NDVI_min+NDVI_max+NDVI_mean+NDVI_range+NDVI_med+SAVI_min+SAVI_max+SAVI_mean+SAVI_range+SAVI_med'

In [206]:
start_time = timeit.default_timer() 
model_log = smf.glm(formula = formula, data=df, family=sm.families.Binomial())
result = model_log.fit()
print(result.summary())
elapsed = timeit.default_timer() - start_time #gives total computation time
print("---Run time is %s seconds ---" % elapsed) #prints computation time

                              Generalized Linear Model Regression Results                               
Dep. Variable:     ['Veg_class[non-woody]', 'Veg_class[woody]']   No. Observations:                 4339
Model:                                                      GLM   Df Residuals:                     4306
Model Family:                                          Binomial   Df Model:                           32
Link Function:                                            logit   Scale:                          1.0000
Method:                                                    IRLS   Log-Likelihood:                -2474.5
Date:                                          Sun, 28 Nov 2021   Deviance:                       4949.0
Time:                                                  22:18:05   Pearson chi2:                 4.74e+03
No. Iterations:                                               9                                         
Covariance Type:                                      n

### Next I use the smf.glm() function to run the linear regression model to see if it can handle more variables compared to sklearn. It turns out that the smf.glm() module can handle more variables (can include CH_max, CH_range, CC_min, CC_max, and CC_range), so I will use this moving forward. 

### The smallest p-value here is found for the CH_max variable, with a p-value of 0.056. This is not below the 0.05 significance threshold, therefore we conclude that none of the independent variables have a significant relationship with the Veg_Class variable based on the logistic regression model. 

### The p-value of the CH_max variable is very close to having a relationship with Veg_Class, however, and its negative coefficient indicates that non-woody cover (closer to a value of 1.0) may be more likely to have a lower maximum canopy height.  


### We also calculate the model runtime as 0.0933 seconds

In [207]:
predictions = result.predict()
print(predictions[0:10])

[0.16866834 0.14658174 0.65134383 0.61338492 0.59084199 0.25340254
 0.36703228 0.87421934 0.12872417 0.12385194]


### Here we use the predict() function to predict the probability that the ground cover is non-woody, given the values of all predictors. Since there is no dataset provided for the predict() function, probabilities are calculated using the entire dataset. 

### The first ten probabilities of non-woody cover are printed above

In [208]:
print(np.column_stack((df.loc[:,"Veg_class"], result.model.endog))) # np.vstack or np.hstack
# exog means x-values; and endog means Y-values

[['woody' 0.0]
 ['non-woody' 1.0]
 ['non-woody' 1.0]
 ...
 ['woody' 0.0]
 ['woody' 0.0]
 ['woody' 0.0]]


### Above, we see that 'non-woody' has been mapped to 1.0 and 'woody' has been mapped to 0.0 by Python

In [209]:
predictions_nominal = ["woody" if x < 0.5 else "non-woody" for x in predictions]
print(predictions_nominal[0:10])

['woody', 'woody', 'non-woody', 'non-woody', 'non-woody', 'woody', 'woody', 'non-woody', 'woody', 'woody']


### Above, we set a condition to assign class labels to the predicted probabilities of non-woody cover. If the probability of non-woody cover is less than 0.5, the value is assigned to woody, otherwise it is assigned to non-woody. Cross referencing the predicted probability values with the list above lets us know that the labeling is correct.

### Now that we have relabeled our probabilities, we can create a confusion matrix to assess our model performance

In [210]:
from sklearn.metrics import confusion_matrix, classification_report
print(confusion_matrix(df["Veg_class"], 
                       predictions_nominal))

[[1487  601]
 [ 736 1515]]


### The values going diagonally from top left to bottom right indicate correctly classified records. The other values are instances where the model either predicted a false positive or false negative response. 

### Our model correctly classified vegetation cover 2254 + 726 = 2980 times out of a total 3869 records, yielding an overall accuracy of 77.0%.

### Our model correctly identified 2254/2510 non-woody records, yielding a sensitivity value of 89.8%.

### Our model correctly identified 726/1359 woody records, yielding a specificity value of 53.4%.

### Overall accuracy and other performance metrics such as precision, recall, and f1-score can be checked by using the classification_report function below.

In [6]:
df["Veg_class"].value_counts()

woody        2251
non-woody    2088
Name: Veg_class, dtype: int64

In [212]:
print(classification_report(df["Veg_class"], 
                            predictions_nominal, 
                            digits = 3))

              precision    recall  f1-score   support

   non-woody      0.669     0.712     0.690      2088
       woody      0.716     0.673     0.694      2251

    accuracy                          0.692      4339
   macro avg      0.692     0.693     0.692      4339
weighted avg      0.693     0.692     0.692      4339



### At first it seems that our model is predicting woody and non-woody cover with decent accuracy (above 70%), but we must remember that this model was trained and tested on the entire dataset, which leads to over-estimation of the training error rate and under-estimation of the test error rate. 

  

### To get a better assessment of model accuracy, we must split the dataset into a training set and a test set. We will do this using 50% of the original dataset as training data, and the other 50% as test data. This way we can see how well the model classifies new data that have not been used for training.

In [46]:
#begin by shuffling records so that datasets are randomly sampled
#df = shuffle(df)

#next, set 50% of records for training and 50% for test (4339/2 = 2169.5, rounded so that training will include one more record compared to test)
x_train = df[:2170][:]
y_train = df[:2170]['Veg_class']

x_test = df[2170:][:]
y_test = df[2170:]['Veg_class']

In [32]:
start_time = timeit.default_timer() 
model = smf.glm(formula = formula2, 
                data = x_train, 
                family = sm.families.Binomial())
result = model.fit()
print(result.summary())
elapsed = timeit.default_timer() - start_time #gives total computation time
print("---Run time is %s seconds ---" % elapsed) #prints computation time

                 Generalized Linear Model Regression Results                  
Dep. Variable:              Veg_class   No. Observations:                 2170
Model:                            GLM   Df Residuals:                     2140
Model Family:                Binomial   Df Model:                           29
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -1284.3
Date:                Sun, 28 Nov 2021   Deviance:                       2568.7
Time:                        14:07:43   Pearson chi2:                 2.13e+03
No. Iterations:                    10                                         
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -7.5873      2.233     -3.398      0.0

### Strangely, after randomizing and splitting the data into 50% training and 50% test, the model begins to produce warnings about data types, division by 0, and more. In reponse I removed more variables until the model ran without errors.

### Excluded variables include CH_max and CH_range. Unfortunately I had to remove the CH_max and CC_min variables, which showed to have the closest relationship to Veg_Class.

### From this analysis, we see that EVI_range, EVI_min, and EVI_max have the lowest p-values with 0.053. Thus, we conclude that none of the variables have a significant relationship with the response variable according to this logistic regression model.

### Finally, we see that this model took 0.0716 seconds to run.

### Next, I generate model assessment metrics such as precision, recall, and f1-score.

In [47]:
predictions = result.predict(x_test)
predictions_nominal = [ "woody" if x < 0.5 else "non-woody" for x in predictions]
print(classification_report(y_test, 
                            predictions_nominal, 
                            digits = 3))

              precision    recall  f1-score   support

   non-woody      0.661     0.728     0.693      1017
       woody      0.736     0.671     0.702      1152

    accuracy                          0.698      2169
   macro avg      0.699     0.699     0.697      2169
weighted avg      0.701     0.698     0.698      2169



### After running the model using half of the data for training and the other half for testing, we see that overall accuracy decreased from 77% to 75.9%. 

### Sensitivity decreased from 89.8% to 87.8%

### Specificity increased from 53.4% to 54.3%

In [48]:
print(confusion_matrix(y_test, predictions_nominal))

[[740 277]
 [379 773]]


# 5-Fold Cross Validation for Logistic Regression

### Now that we have run the Logistic Regression model using 100% of data for training/test, we will use 5-fold cross validation to futher assess our model performance.

In [10]:
kf = KFold(n_splits=5, random_state=42, shuffle=True) #initiate 5-fold cross, with shuffling and random state = 42
start_time = timeit.default_timer() #defines start time


model_log = LogisticRegression(solver= 'liblinear')

acc_score = [];
Truth = [];
output_pred = []; # set up empty lists to store metrics
output_ID = [];
index = [];

for train_index , test_index in kf.split(df):
    #print(train_index); ## to check the training index
    #print(test_index); ## to check the testing index
    #print(); print();
    
    X_train , X_test, X_full = df.iloc[train_index, 5:-1], df.iloc[test_index, 5:-1], df.iloc[test_index, :-1] #set training and test data for x
    y_train , y_test = df.iloc[train_index,-1], df.iloc[test_index,-1] #set training and test data for y
    
    output_ID.append(X_full['Id'].tolist())
    
    #print([X_train.shape, y_train.shape]);
    #print([X_test.shape, y_test.shape]);
    
    model_log.fit(X_train,y_train) #fit model using x training records
    pred_values = model_log.predict(X_test) #generate predicted values based on x test records
    
    output_pred.append(pred_values.tolist())
    
    acc = accuracy_score(y_test, pred_values) #use training vs test to get accuracy
    acc_score.append(acc) #append accuracy value to list
    
    Truth.extend(y_test.values.reshape(y_test.shape[0])); ## it is a list 

elapsed = timeit.default_timer() - start_time #gives total computation time
print("---Run time is %s seconds ---" % elapsed) #prints computation time
print()

print('Accuracy of each fold: \n {}'.format(acc_score))
print()
print('Avg accuracy : \n{}'.format(np.mean(acc_score))); 
print()
print('Std of accuracy : \n{}'.format(np.std(acc_score)))

output_IDFlat = [item for sublist in output_ID for item in sublist]
output_predFlat = [item for sublist in output_pred for item in sublist]

predictionDF = pd.DataFrame(list(zip(output_IDFlat, output_predFlat)), columns = ['ID', 'Prediction'])

predictionDF.to_csv("Log_Reg_pred.csv")

---Run time is 0.22666449999996985 seconds ---

Accuracy of each fold: 
 [0.6762672811059908, 0.6797235023041475, 0.6831797235023042, 0.6670506912442397, 0.7151095732410612]

Avg accuracy : 
0.6842661542795487

Std of accuracy : 
0.01633087047040362


### Average accuracy = 69.3%
### Standard deviation = 0.015%

## Next, we generate a confusion matrix for the 5-fold cross to get a more detailed look at model performance

In [13]:
## Convert a list to an array
Truth = np.asarray(Truth)  ## or np.array(Truth)
Output = np.asarray(output_predFlat)

np.column_stack((Truth, Output))

print(classification_report(Truth, Output))

              precision    recall  f1-score   support

   non-woody       0.66      0.73      0.69      2088
       woody       0.72      0.65      0.68      2251

    accuracy                           0.68      4339
   macro avg       0.69      0.69      0.68      4339
weighted avg       0.69      0.68      0.68      4339



### Overall accuracy = 69%
### Sensitivity = 70%
### Specificity = 69%
### Runtime = 0.3639 sec.

### 5-fold cross shows very similar accuracy metrics compared to using 100% and 50% of data for testing. 5-fold cross had slightly higher specificity, indicating that the woody class is being predicted with slightly more accuracy. 5-fold cross validation also took more time to run than running the model itself, but when factoring in time to set up the validation set method, the 5-fold cross validation may save time.

## Part 2: K-Nearest Neighbors (KNN)

### In this section I will use the KNN classifier with various K values to perform land cover classification. One model will be run with the entire dataset as training and test and one model will be run with 50% of the dataset used for training and 50% of the dataset used for test. K values 1-10 will be tested for each model to select the ideal K value. 

In [51]:
#first I remap the woody and non-woody variables to 0 = non-woody and 1 = woody
df.Veg_class = pd.Categorical(df.Veg_class)
df['Veg_class'] = df.Veg_class.cat.codes
df.head()


#Next I run the KNN model using the entire dataset for training and test, with k-values 1-10
start_time = timeit.default_timer() 
for k in range(1,11):
    knn = neighbors.KNeighborsClassifier(n_neighbors=k)
    pred = knn.fit(df[['CH_min', 'CH_max', 'CH_mean', 'CH_range', 'CH_med', 'CC_min', 'CC_max', 'CC_mean', 'CC_range', 'CC_med', 'CD_min', 'CD_max', 'CD_mean', 'CD_range', 'CD_med', 'ARVI_min', 'ARVI_max', 'ARVI_mean', 'ARVI_range', 'ARVI_med', 'EVI_min', 'EVI_max', 'EVI_mean', 'EVI_range', 'EVI_med', 'NDVI_min', 'NDVI_max', 'NDVI_mean', 'NDVI_range', 'NDVI_med', 'SAVI_min', 'SAVI_max', 'SAVI_mean', 'SAVI_range', 'SAVI_med']], df['Veg_class']).predict(df[['CH_min', 'CH_max', 'CH_mean', 'CH_range', 'CH_med', 'CC_min', 'CC_max', 'CC_mean', 'CC_range', 'CC_med', 'CD_min', 'CD_max', 'CD_mean', 'CD_range', 'CD_med', 'ARVI_min', 'ARVI_max', 'ARVI_mean', 'ARVI_range', 'ARVI_med', 'EVI_min', 'EVI_max', 'EVI_mean', 'EVI_range', 'EVI_med', 'NDVI_min', 'NDVI_max', 'NDVI_mean', 'NDVI_range', 'NDVI_med', 'SAVI_min', 'SAVI_max', 'SAVI_mean', 'SAVI_range', 'SAVI_med']])
    
    
    print("k is", k)
    print(confusion_matrix(df['Veg_class'], pred).T)
    print(classification_report(df['Veg_class'], pred, digits=3))
elapsed = timeit.default_timer() - start_time #gives total computation time
print("---Run time is %s seconds ---" % elapsed) #prints computation time

k is 1
[[2088    0]
 [   0 2251]]
              precision    recall  f1-score   support

           0      1.000     1.000     1.000      2088
           1      1.000     1.000     1.000      2251

    accuracy                          1.000      4339
   macro avg      1.000     1.000     1.000      4339
weighted avg      1.000     1.000     1.000      4339

k is 2
[[2088  928]
 [   0 1323]]
              precision    recall  f1-score   support

           0      0.692     1.000     0.818      2088
           1      1.000     0.588     0.740      2251

    accuracy                          0.786      4339
   macro avg      0.846     0.794     0.779      4339
weighted avg      0.852     0.786     0.778      4339

k is 3
[[1688  490]
 [ 400 1761]]
              precision    recall  f1-score   support

           0      0.775     0.808     0.791      2088
           1      0.815     0.782     0.798      2251

    accuracy                          0.795      4339
   macro avg      0.795   

### k = 3 performs best (best tradeoff between metrics)
### overall accuracy = 79.5%
### sensitivity = 80.8%
### specificity = 78.2%
### runtime = 2.0658

In [53]:
#Next I run the KNN model using 50% of the data for training and 50% of the data for testing

x_train = df.iloc[:2170, :-1]
y_train = df[:2170]['Veg_class']

x_test = df.iloc[2170:, :-1]
y_test = df[2170:]['Veg_class']

start_time = timeit.default_timer() 
for k in range(1,11):
    knn = neighbors.KNeighborsClassifier(n_neighbors=k)
    pred = knn.fit(x_train, y_train).predict(x_test)
    
    
    print("k is", k)
    print(confusion_matrix(y_test, pred).T)
    print(classification_report(y_test, pred, digits=3))
elapsed = timeit.default_timer() - start_time #gives total computation time
print("---Run time is %s seconds ---" % elapsed) #prints computation time

k is 1
[[551 536]
 [466 616]]
              precision    recall  f1-score   support

           0      0.507     0.542     0.524      1017
           1      0.569     0.535     0.551      1152

    accuracy                          0.538      2169
   macro avg      0.538     0.538     0.538      2169
weighted avg      0.540     0.538     0.538      2169

k is 2
[[789 776]
 [228 376]]
              precision    recall  f1-score   support

           0      0.504     0.776     0.611      1017
           1      0.623     0.326     0.428      1152

    accuracy                          0.537      2169
   macro avg      0.563     0.551     0.520      2169
weighted avg      0.567     0.537     0.514      2169

k is 3
[[539 538]
 [478 614]]
              precision    recall  f1-score   support

           0      0.500     0.530     0.515      1017
           1      0.562     0.533     0.547      1152

    accuracy                          0.532      2169
   macro avg      0.531     0.531     

In [61]:
print(y_train.shape)

(3472,)


### k = 5 performs best (best tradeoff between three metrics)
### overall accuracy = 54.3%
### sensitivity = 53.6%
### specificity = 54.9%
### runtime = 0.8780 seconds

## 5-fold Cross Validation
### 5-fold cross validation will be run with k values 1-10 as in the validation set method to assess each k value.

In [16]:
kf = KFold(n_splits=5, random_state=42, shuffle=True)

result=[]
start_time = timeit.default_timer() 
for k in range(1,11):
    
    knn = neighbors.KNeighborsClassifier(n_neighbors = k)
    
    acc_score = [];
    Truth = [];
    output_pred = [];
    output_ID = [];

    for train_index , test_index in kf.split(df):
        #print(train_index); ## to check the training index
        #print(test_index); ## to check the testing index
        #print(); print();
    
        X_train , X_test, X_full = df.iloc[train_index, 5:-1], df.iloc[test_index, 5:-1], df.iloc[test_index, :-1]
        y_train , y_test = df.iloc[train_index,-1], df.iloc[test_index,-1]
    
        #print([X_train.shape, y_train.shape]);
        #print([X_test.shape, y_test.shape]);
        
        output_ID.append(X_full['Id'].tolist())
        
        knn.fit(X_train,y_train)
        pred_values = knn.predict(X_test)
        
        output_pred.append(pred_values.tolist())
     
        acc = accuracy_score(y_test, pred_values)
        acc_score.append(acc)
        
        
        Truth.extend(y_test.values.reshape(y_test.shape[0])); ## it is a list 
    
    print("k is", k)
    print('Accuracy of each fold: \n {}'.format(acc_score))
    print()
    print('Avg accuracy : \n{}'.format(np.mean(acc_score))); 
    print()
    print('Std of accuracy : \n{}'.format(np.std(acc_score)))

    output_IDFlat = [item for sublist in output_ID for item in sublist]
    output_predFlat = [item for sublist in output_pred for item in sublist]
    
    predictionDF = pd.DataFrame(list(zip(output_IDFlat, output_predFlat)), columns = ['ID', 'Prediction'])
    
    predictionDF.to_csv('knn'+str(k)+'_pred.csv')
    
    Truth = np.asarray(Truth)  ## or np.array(Truth)
    Output = np.asarray(output_predFlat)

    np.column_stack((Truth, Output))

    print(classification_report(Truth, Output))
    
elapsed = timeit.default_timer() - start_time #gives total computation time
print("---Run time is %s seconds ---" % elapsed) #prints computation time
print()

k is 1
Accuracy of each fold: 
 [0.597926267281106, 0.586405529953917, 0.565668202764977, 0.5737327188940092, 0.5940023068050749]

Avg accuracy : 
0.5835470051398168

Std of accuracy : 
0.012167328923190284
              precision    recall  f1-score   support

   non-woody       0.57      0.58      0.57      2088
       woody       0.60      0.58      0.59      2251

    accuracy                           0.58      4339
   macro avg       0.58      0.58      0.58      4339
weighted avg       0.58      0.58      0.58      4339

k is 2
Accuracy of each fold: 
 [0.5806451612903226, 0.5806451612903226, 0.5817972350230415, 0.5576036866359447, 0.5836216839677048]

Avg accuracy : 
0.5768625856414673

Std of accuracy : 
0.009690790707536317
              precision    recall  f1-score   support

   non-woody       0.54      0.81      0.65      2088
       woody       0.67      0.36      0.47      2251

    accuracy                           0.58      4339
   macro avg       0.61      0.59     

### k = 10 has the highest overall accuracy with 55%. This is well below the accuracy reported for the validation set method.

### Runtime = 3.0314 seconds

## Part 3: Linear Discriminant Analysis (LDA)


### In this section I will be using Linear Discriminant Analysis for land cover classification

### As with the other models, one model will be trained and tested with the entire dataset and one model will be trained with 50% of the dataset and tested with 50% of the dataset

In [55]:
#start with using the entire dataset for training and test
start_time = timeit.default_timer() 
lda = LinearDiscriminantAnalysis()
model_lda = lda.fit(df[['CH_min', 'CH_max', 'CH_mean', 'CH_range', 'CH_med', 'CC_min', 'CC_max', 'CC_mean', 'CC_range', 'CC_med', 'CD_min', 'CD_max', 'CD_mean', 'CD_range', 'CD_med', 'ARVI_min', 'ARVI_max', 'ARVI_mean', 'ARVI_range', 'ARVI_med', 'EVI_min', 'EVI_max', 'EVI_mean', 'EVI_range', 'EVI_med', 'NDVI_min', 'NDVI_max', 'NDVI_mean', 'NDVI_range', 'NDVI_med', 'SAVI_min', 'SAVI_max', 'SAVI_mean', 'SAVI_range', 'SAVI_med']], df['Veg_class'])

print(model_lda.priors_)
print(model_lda.means_)
print(model_lda.coef_)

pred=model_lda.predict(df[['CH_min', 'CH_max', 'CH_mean', 'CH_range', 'CH_med', 'CC_min', 'CC_max', 'CC_mean', 'CC_range', 'CC_med', 'CD_min', 'CD_max', 'CD_mean', 'CD_range', 'CD_med', 'ARVI_min', 'ARVI_max', 'ARVI_mean', 'ARVI_range', 'ARVI_med', 'EVI_min', 'EVI_max', 'EVI_mean', 'EVI_range', 'EVI_med', 'NDVI_min', 'NDVI_max', 'NDVI_mean', 'NDVI_range', 'NDVI_med', 'SAVI_min', 'SAVI_max', 'SAVI_mean', 'SAVI_range', 'SAVI_med']])
print(np.unique(pred, return_counts=True))
print(confusion_matrix(df['Veg_class'], pred))
print()
print(classification_report(df['Veg_class'], pred, digits=3))
elapsed = timeit.default_timer() - start_time #gives total computation time
print("---Run time is %s seconds ---" % elapsed) #prints computation time

[0.48121687 0.51878313]
[[5.58429119e-03 4.30028734e-02 1.49203417e-02 3.74185822e-02
  1.30531609e-02 1.54677202e+01 6.45607274e+01 3.82010689e+01
  4.90930073e+01 3.72941329e+01 1.54716953e+01 6.45329975e+01
  3.81849045e+01 4.90613023e+01 3.72730840e+01 3.81274211e-02
  1.85078672e-01 9.94291988e-02 1.46951251e-01 9.30927518e-02
  1.43241749e-01 2.11871499e-01 1.71893421e-01 6.86297506e-02
  1.68902770e-01 2.10016125e-01 3.34135376e-01 2.62371235e-01
  1.24119250e-01 2.57300461e-01 1.42705280e-01 2.07888906e-01
  1.70312378e-01 6.51836286e-02 1.67678809e-01]
 [5.17236783e-02 2.73025321e-01 1.43306035e-01 2.21301644e-01
  1.29386939e-01 1.95057306e+01 7.03702349e+01 4.42549972e+01
  5.08645044e+01 4.39452463e+01 1.93281651e+01 6.99373607e+01
  4.38089342e+01 5.06091957e+01 4.34224342e+01 1.10782208e-01
  3.03764405e-01 2.06690971e-01 1.92982196e-01 2.05608390e-01
  1.77020225e-01 2.65746094e-01 2.20985702e-01 8.87258674e-02
  2.20464308e-01 2.74197700e-01 4.31125843e-01 3.52945620e-0

### Overall accuracy = 68.8%
### Sensitivity = 70.4%
### Specificity = 67.3%
### runtime = 0.02990 seconds

In [56]:
#Next, use 50% of data for training and 50% for test
start_time = timeit.default_timer() 
lda = LinearDiscriminantAnalysis()
model_lda = lda.fit(x_train, y_train)

print(model_lda.priors_)
print(model_lda.means_)
print(model_lda.coef_)

pred=model_lda.predict(x_test)
print(np.unique(pred, return_counts=True))
print(confusion_matrix(y_test, pred))
print()
print(classification_report(y_test, pred, digits=3))
elapsed = timeit.default_timer() - start_time #gives total computation time
print("---Run time is %s seconds ---" % elapsed) #prints computation time

ValueError: Found input variables with inconsistent numbers of samples: [2170, 3472]

### Overall accuracy = 77.6%
### Sensitivity = 90.5%
### Specificity = 54.1%
### runtime = 0.0489 seconds

## 5-fold Cross Validation
### Next, we use the 5-fold cross validation method to assess LDA model performance.

In [17]:
start_time = timeit.default_timer()
kf = KFold(n_splits=5, random_state=42, shuffle=True)

lda = LinearDiscriminantAnalysis()


acc_score = [];
Truth = [];
output_pred = [];
output_ID = [];

for train_index , test_index in kf.split(df):
    #print(train_index); ## to check the training index
    #print(test_index); ## to check the testing index
    #print(); print();
    
    X_train , X_test, X_full = df.iloc[train_index, 5:-1], df.iloc[test_index, 5:-1], df.iloc[test_index, :-1]
    y_train , y_test = df.iloc[train_index,-1], df.iloc[test_index,-1]
    
    output_ID.append(X_full['Id'].tolist())
    
    #print([X_train.shape, y_train.shape]);
    #print([X_test.shape, y_test.shape]);
    
    lda.fit(X_train,y_train)
    pred_values = lda.predict(X_test)
    
    output_pred.append(pred_values.tolist())
     
    acc = accuracy_score(y_test, pred_values)
    acc_score.append(acc)
    
    Truth.extend(y_test.values.reshape(y_test.shape[0])); ## it is a list 

elapsed = timeit.default_timer() - start_time #gives total computation time
print("---Run time is %s seconds ---" % elapsed) #prints computation time
print()

print('Accuracy of each fold: \n {}'.format(acc_score))
print()
print('Avg accuracy : \n{}'.format(np.mean(acc_score))); 
print()
print('Std of accuracy : \n{}'.format(np.std(acc_score)))

output_IDFlat = [item for sublist in output_ID for item in sublist]
output_predFlat = [item for sublist in output_pred for item in sublist]

predictionDF = pd.DataFrame(list(zip(output_IDFlat, output_predFlat)), columns = ['ID', 'Prediction'])
predictionDF.to_csv('LDA_pred.csv')

Truth = np.asarray(Truth)  ## or np.array(Truth)
Output = np.asarray(output_predFlat)

np.column_stack((Truth, Output))

print(classification_report(Truth, Output))

---Run time is 0.10089260000040667 seconds ---

Accuracy of each fold: 
 [0.6670506912442397, 0.6797235023041475, 0.6762672811059908, 0.6751152073732719, 0.7104959630911188]

Avg accuracy : 
0.6817305290237536

Std of accuracy : 
0.014970541399163
              precision    recall  f1-score   support

   non-woody       0.66      0.70      0.68      2088
       woody       0.70      0.66      0.68      2251

    accuracy                           0.68      4339
   macro avg       0.68      0.68      0.68      4339
weighted avg       0.68      0.68      0.68      4339



## Part 4: Quadratic Discriminant Analysis (QDA)

### In this section I will be using Quadratice Discriminant Analysis for land cover classification

### As with the other models, one model will be trained and tested with the entire dataset and one model will be trained with 50% of the dataset and tested with 50% of the dataset

In [29]:
#start with using the entire dataset for training and test
start_time = timeit.default_timer() 
qda = QuadraticDiscriminantAnalysis()
model_qda = qda.fit(df[['CH_min', 'CH_max', 'CH_mean', 'CH_range', 'CH_med', 'CC_min', 'CC_max', 'CC_mean', 'CC_range', 'CC_med', 'CD_min', 'CD_max', 'CD_mean', 'CD_range', 'CD_med', 'ARVI_min', 'ARVI_max', 'ARVI_mean', 'ARVI_range', 'ARVI_med', 'EVI_min', 'EVI_max', 'EVI_mean', 'EVI_range', 'EVI_med', 'NDVI_min', 'NDVI_max', 'NDVI_mean', 'NDVI_range', 'NDVI_med', 'SAVI_min', 'SAVI_max', 'SAVI_mean', 'SAVI_range', 'SAVI_med']], df['Veg_Class'])
print(model_qda.priors_)
print()
print(model_qda.means_)

pred2=model_qda.predict(df[['CH_min', 'CH_max', 'CH_mean', 'CH_range', 'CH_med', 'CC_min', 'CC_max', 'CC_mean', 'CC_range', 'CC_med', 'CD_min', 'CD_max', 'CD_mean', 'CD_range', 'CD_med', 'ARVI_min', 'ARVI_max', 'ARVI_mean', 'ARVI_range', 'ARVI_med', 'EVI_min', 'EVI_max', 'EVI_mean', 'EVI_range', 'EVI_med', 'NDVI_min', 'NDVI_max', 'NDVI_mean', 'NDVI_range', 'NDVI_med', 'SAVI_min', 'SAVI_max', 'SAVI_mean', 'SAVI_range', 'SAVI_med']])
print(np.unique(pred2, return_counts=True))
print()
print(confusion_matrix(df['Veg_Class'], pred2))
print()
print(classification_report(df['Veg_Class'], pred2, digits=3))
elapsed = timeit.default_timer() - start_time #gives total computation time
print("---Run time is %s seconds ---" % elapsed) #prints computation time

[0.64874645 0.35125355]

[[4.05976096e-03 2.75019920e-02 1.25140943e-02 2.34422310e-02
  1.16035857e-02 1.18697210e+01 6.90014336e+01 3.82073677e+01
  5.71317127e+01 3.69702786e+01 1.18697210e+01 6.89881667e+01
  3.82050500e+01 5.71184458e+01 3.69702786e+01 1.88597495e-02
  1.89581977e-01 9.19690975e-02 1.70722228e-01 8.40892780e-02
  1.31612837e-01 2.08867981e-01 1.65098788e-01 7.72551447e-02
  1.62000046e-01 1.91405354e-01 3.36960050e-01 2.54430154e-01
  1.45554698e-01 2.48124194e-01 1.30994836e-01 2.05366027e-01
  1.63560773e-01 7.43711922e-02 1.60757318e-01]
 [1.46725534e-02 1.98565122e-01 8.27510361e-02 1.83892569e-01
  6.32008829e-02 1.31161147e+01 7.45027956e+01 4.27195312e+01
  6.13866811e+01 4.22453272e+01 1.31247239e+01 7.44329648e+01
  4.26608627e+01 6.13082411e+01 4.21465045e+01 7.93598447e-02
  3.22179671e-01 2.01130450e-01 2.42819828e-01 2.01333353e-01
  1.56951563e-01 2.65500199e-01 2.10556075e-01 1.08548637e-01
  2.09937252e-01 2.43662453e-01 4.44557441e-01 3.45535444e-

C:\Users\tsh3146\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:715: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


### Overall accuracy = 69.6%
### Sensitivity = 94.8%
### Specificity = 23.0%
### runtime = 0.0818 seconds

In [30]:
#Next, use 50% of data for training and 50% for test

start_time = timeit.default_timer() 
qda = QuadraticDiscriminantAnalysis()
model_qda = qda.fit(x_train, y_train)
print(model_qda.priors_)
print()
print(model_qda.means_)

pred2=model_qda.predict(x_test)
print(np.unique(pred2, return_counts=True))
print()
print(confusion_matrix(y_test, pred2))
print()
print(classification_report(y_test, pred2, digits=3))
elapsed = timeit.default_timer() - start_time #gives total computation time
print("---Run time is %s seconds ---" % elapsed) #prints computation time

[0.65356774 0.34643226]

[[2.04970728e+03 2.04970728e+03 2.04970728e+03 2.81977848e+01
  8.32041139e+00 4.12183544e-03 2.90031645e-02 1.25997495e-02
  2.48813290e-02 1.16178797e-02 1.19321993e+01 6.91114709e+01
  3.83908637e+01 5.71792717e+01 3.74227450e+01 1.19321993e+01
  6.91114709e+01 3.83910220e+01 5.71792717e+01 3.74227450e+01
  1.67604952e-02 1.87984325e-01 8.93671421e-02 1.71223830e-01
  8.12018562e-02 1.30882665e-01 2.08438983e-01 1.64271732e-01
  7.75563174e-02 1.60925972e-01 1.89702169e-01 3.35690157e-01
  2.52361068e-01 1.45987989e-01 2.45843203e-01 1.30306447e-01
  2.05016497e-01 1.62776865e-01 7.47100510e-02 1.59718646e-01]
 [1.84577313e+03 1.84577313e+03 1.84577313e+03 2.10797015e+01
  5.98007463e+00 1.63283582e-02 2.20477612e-01 9.12068627e-02
  2.04149254e-01 6.80298507e-02 1.30697014e+01 7.41992532e+01
  4.27603303e+01 6.11295519e+01 4.24770147e+01 1.30573133e+01
  7.42241786e+01 4.27086651e+01 6.11668654e+01 4.23286565e+01
  7.61676004e-02 3.23095016e-01 2.00441605e-

C:\Users\tsh3146\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:715: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


### Overall accuracy = 68.8%
### Sensitivity = 99.6%
### Specificity = 13.1%
### runtime = 0.0503 seconds

In [21]:
start_time = timeit.default_timer()
kf = KFold(n_splits=5, random_state=42, shuffle=True)

qda = QuadraticDiscriminantAnalysis()

acc_score = [];
Truth = [];
output_pred = [];
output_ID = [];

for train_index , test_index in kf.split(df):
    #print(train_index); ## to check the training index
    #print(test_index); ## to check the testing index
    #print(); print();
    
    X_train , X_test, X_full = df.iloc[train_index, 5:-1], df.iloc[test_index, 5:-1], df.iloc[test_index, :-1]
    y_train , y_test = df.iloc[train_index,-1], df.iloc[test_index,-1]
    
    output_ID.append(X_full['Id'].tolist())
    
    #print([X_train.shape, y_train.shape]);
    #print([X_test.shape, y_test.shape]);
    
    qda.fit(X_train,y_train)
    pred_values = qda.predict(X_test)
    
    output_pred.append(pred_values.tolist())
     
    acc = accuracy_score(y_test, pred_values)
    acc_score.append(acc)
    
    Truth.extend(y_test.values.reshape(y_test.shape[0])); ## it is a list 
    
elapsed = timeit.default_timer() - start_time #gives total computation time
print("---Run time is %s seconds ---" % elapsed) #prints computation time
print()

print('Accuracy of each fold: \n {}'.format(acc_score))
print()
print('Avg accuracy : \n{}'.format(np.mean(acc_score))); 
print()
print('Std of accuracy : \n{}'.format(np.std(acc_score)))

output_IDFlat = [item for sublist in output_ID for item in sublist]
output_predFlat = [item for sublist in output_pred for item in sublist]

predictionDF = pd.DataFrame(list(zip(output_IDFlat, output_predFlat)), columns = ['ID', 'Prediction'])
predictionDF.to_csv('QDA_pred.csv')

Truth = np.asarray(Truth)  ## or np.array(Truth)
Output = np.asarray(output_predFlat)

np.column_stack((Truth, Output))

print(classification_report(Truth, Output))

---Run time is 0.05059780000010505 seconds ---

Accuracy of each fold: 
 [0.5679723502304147, 0.5645161290322581, 0.5956221198156681, 0.5253456221198156, 0.5732410611303345]

Avg accuracy : 
0.5653394564656982

Std of accuracy : 
0.022744633825008704
              precision    recall  f1-score   support

   non-woody       0.53      0.95      0.68      2088
       woody       0.83      0.20      0.33      2251

    accuracy                           0.57      4339
   macro avg       0.68      0.58      0.50      4339
weighted avg       0.68      0.57      0.50      4339



C:\Users\tsh3146\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:715: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\tsh3146\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:715: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\tsh3146\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:715: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\tsh3146\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:715: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\tsh3146\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:715: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
